In [ ]:
main_path = 'course-material-time-series-forecasting-product'
nyc_data = main_path+str("/nyc_data.csv")
print(nyc_data)

In [ ]:
# Libraries 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid

In [ ]:
# load the data
# YYYY-MM-DD
df = pd.read_csv(nyc_data, index_col=0, parse_dates=True)
df.head() 

In [ ]:
# Rename variable 
df = df.rename(columns={'Demand': 'y'})
df.head(0)

In [ ]:
# Extract regressors (Easter up until the end)
X = df.iloc[:, 1:]
X.head(0)

# Stationarity 

In [ ]:
# Test 
from statsmodels.tsa.stattools import adfuller 
# x is our data series
# check the returns of this function. p-value is at index 1. 
pvalue = adfuller(x=df.y)[1]

# condition to read test 
if pvalue < 0.05: 
    print(f'Time Series is stationary.\np-value is: {pvalue}')
else: 
    print(f'Time Series is NOT stationary.\np-value is: {pvalue}')

In [ ]:
# Because the p-value is higher than 0.05 and the series isnt stationary, we need to do differencing. 
df.y.diff().dropna()

In [ ]:
# Test 
from statsmodels.tsa.stattools import adfuller 
# x is our data series
# check the returns of this function. p-value is at index 1. 
pvalue = adfuller(x=df.y.diff().dropna())[1]

# condition to read test 
if pvalue < 0.05: 
    print(f'Time Series is stationary.\np-value is: {pvalue}')
else: 
    print(f'Time Series is NOT stationary.\np-value is: {pvalue}')

# Sarimax model

In [ ]:
import pmdarima as pm
from pmdarima import model_selection

In [ ]:
# Model 
# hourly: 24, daily: 7, weekly: 52, monthly:12, quarterly:4
# You can only choose to have one seasonality when using ARIMA/SARIMAX
# For daily data we use 7. 
model = pm.ARIMA(
    X = X,
    order = (1,1,1),
    seasonal_order= (1,1,1,7),
    # Sometimes some combinations of parameters give 'impossible statistical values' and the code stops. 
    # We can supress the warning otherwise once this problem is encountered, the code stops. 
    suppress_warnings= True,  
    # default=True. False for two reasons. 1: less erros. 2: it is good to release constrains when doing the parameter tunning. 
    force_stationarity = False
    )

In [26]:
# Cross-validation
cv = model_selection.RollingForecastCV(
    h=31, # h parameter = horizon = periods we want to predict in the future
    step=16, #cycle of the forecasting ???
    initial= df.shape[0]-180 #180 days in the past
    )

# Visualizing the metrics
cv_score = model_selection.cross_val_score(
    model, 
    y=df.y, 
    scoring='mean_squared_error',
    cv = cv, 
    verbose = 2, 
    error_score = 10000000000000000000000 #??? I do not understand this parameter yet
)

[CV] fold=0 ..........................................................
[CV] fold=1 ..........................................................
[CV] fold=2 ..........................................................
[CV] fold=3 ..........................................................
[CV] fold=4 ..........................................................
[CV] fold=5 ..........................................................
[CV] fold=6 ..........................................................
[CV] fold=7 ..........................................................
[CV] fold=8 ..........................................................
[CV] fold=9 ..........................................................


In [29]:
# CV performance (without the parameter tunning)

# mean squared average
np.average(cv_score)

# root mean squared error 
np.sqrt(np.average(cv_score))

59.955301678704274

# Now we can start the parameter tunning

In [33]:
#Grid  (usually, lower values are the optimal solution, so ideally it is good to start with low values for tunning)
param_grid = {
    'p':[0,1],
    'd':[1],
    'q':[0,1],
    'P':[0,1],
    'D':[0,1],
    'Q':[0,1],
    }

grid = ParameterGrid(param_grid)
len(list(grid))

32